In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import datetime

In [2]:
from Optimizers import cauchy_simplex, egd, buy_and_hold
from Optimizers.utils import best_returns

In [3]:
import os

In [4]:
def sharpe_ratio(price_relatives, risk_free=0.04):
    price_relatives = np.array(price_relatives)
    APY = annualized_percentage_yield(price_relatives)
    
    returns = price_relatives - 1
    return (APY - risk_free) / np.std(returns)

In [5]:
def annualized_percentage_yield(price_relatives):
    price_relatives = np.array(price_relatives)
    
    y = len(price_relatives) // 252
    
    cumulative_wealth = np.prod(price_relatives)
    APY = pow(cumulative_wealth, 1 / y) - 1
    
    return APY

In [6]:
def evaluate_returns(price_relatives, risk_free=0.04):
    return (annualized_percentage_yield(price_relatives), 
            sharpe_ratio(price_relatives, risk_free=risk_free))

In [7]:
def run_cs(returns):
    """ returns assumed to be the price relatives """
    returns = np.array(returns)
    
    T, N = returns.shape

    rescaled_returns = returns / np.max(returns, axis=1)[:, None]
    a = np.min(rescaled_returns)
    
    cs_eta = a * np.sqrt(np.log(N)) / (a * np.sqrt(np.log(N)) + np.sqrt(T))
    _, cs_history = cauchy_simplex(rescaled_returns, cs_eta)
    
    return cs_history

def run_egd(returns):
    """ returns assumed to be the price relatives """
    returns = np.array(returns)
    
    T, N = returns.shape

    rescaled_returns = returns / np.max(returns, axis=1)[:, None]
    a = np.min(rescaled_returns)
    
    egd_eta = 2 * a * np.sqrt(2 * np.log(N) / T)
    _, egd_history = egd(rescaled_returns, egd_eta)

    return egd_history

def run_hold(returns):
    """ returns assumed to be the price relatives """
    returns = np.array(returns)
    
    T, N = returns.shape

    rescaled_returns = returns / np.max(returns, axis=1)[:, None]
    
    _, hold_history = buy_and_hold(rescaled_returns)

    return hold_history

In [8]:
root_dir = 'Datasets/clean/'

markets = [x.replace('.csv', '') for x in os.listdir(root_dir)]
markets

['NYSE', 'DJIA', 'SP500', 'TSE']

In [9]:
apy_results = pd.DataFrame(0.0, index=markets, columns=['CS', 'EGD', 'B&H'])
sharpe_results = pd.DataFrame(0.0, index=markets, columns=['CS', 'EGD', 'B&H'])

optimizer_weights = {key: None for key in apy_results.columns}

for csv_file in os.listdir(root_dir):
    market = csv_file.replace(".csv", "")
    
    returns = pd.read_csv(root_dir + csv_file, index_col=0)
    
    optimizer_weights['CS'] = run_cs(returns)
    optimizer_weights['EGD'] = run_egd(returns)
    optimizer_weights['B&H'] = run_hold(returns)
    
    for optimizer, w_history in optimizer_weights.items():
        running_returns = np.sum(returns * w_history, axis=1)
        apy, sharpe = evaluate_returns(running_returns, risk_free=0.04)
        
        apy_results.loc[market, optimizer] = apy
        sharpe_results.loc[market, optimizer] = sharpe

In [10]:
apy_results

,CS,EGD,B&H
NYSE,0.161767,0.161799,0.129239
DJIA,-0.099426,-0.101220,-0.125723
SP500,0.103890,0.101411,0.060541
TSE,0.123728,0.123485,0.126946


In [11]:
sharpe_results

,CS,EGD,B&H
NYSE,14.359944,14.309884,9.529189
DJIA,-8.714009,-8.848280,-10.811617
SP500,4.594588,4.395150,1.346768
TSE,10.225199,10.204133,10.629036


In [12]:
optimizers = apy_results.columns
markets = apy_results.index

In [13]:
print(r"""\begin{tabular}{l|cc|cc|cc}
\toprule
 & \multicolumn{2}{c|}{CS}&\multicolumn{2}{c|}{EGD} &\multicolumn{2}{c}{B\&H} \\
\midrule
Dataset & APY & Sharpe & APY & Sharpe & APY & Sharpe\\
\midrule""")

for market in apy_results.index:
    apy_values = apy_results.loc[market].values
    sharpe_values = sharpe_results.loc[market].values
    
    min_apy_index, max_apy_index = np.argmin(apy_values), np.argmax(apy_values)
    min_sharpe_index, max_sharpe_index = np.argmin(sharpe_values), np.argmax(sharpe_values)
    
    s = f"{market} "
    for i, (apy, sharpe) in enumerate(zip(apy_values, sharpe_values)):
        if i == min_apy_index:
            s += r"& \underline{" + f"{apy:.3f}" + "} "
        elif i == max_apy_index:
            s += r"& \textbf{" + f"{apy:.3f}" + "} "
        else:
            s += f"& {apy:.3f} "
        
        if i == min_sharpe_index:
            s += r"& \underline{" + f"{sharpe:.3f}" + "} "
        elif i == max_sharpe_index:
            s += r"& \textbf{" + f"{sharpe:.3f}" + "} "
        else:
            s += f"& {sharpe:.3f}"
    
    s += r"\\"
    print(s)

print(r"""\bottomrule
\end{tabular}""")

\begin{tabular}{l|cc|cc|cc}
\toprule
 & \multicolumn{2}{c|}{CS}&\multicolumn{2}{c|}{EGD} &\multicolumn{2}{c}{B\&H} \\
\midrule
Dataset & APY & Sharpe & APY & Sharpe & APY & Sharpe\\
\midrule
NYSE & 0.162 & \textbf{14.360} & \textbf{0.162} & 14.310& \underline{0.129} & \underline{9.529} \\
DJIA & \textbf{-0.099} & \textbf{-8.714} & -0.101 & -8.848& \underline{-0.126} & \underline{-10.812} \\
SP500 & \textbf{0.104} & \textbf{4.595} & 0.101 & 4.395& \underline{0.061} & \underline{1.347} \\
TSE & 0.124 & 10.225& \underline{0.123} & \underline{10.204} & \textbf{0.127} & \textbf{10.629} \\
\bottomrule
\end{tabular}
